# Learning of the Transmitter Orientation via Gradient Descent

This notebook reproduces the results for the example application "Optimization of transmitter orientation" in the paper [Sionna RT: Differentiable Ray Tracing for Radio Propagation Modeling](https://arxiv.org).

It requires [Sionna](https://github.com/NVlabs/sionna) v0.16 or later.


## Imports and GPU Configuration

In [ ]:
# Set some environment variables
import os
gpu_num = "" # GPU to be used. Use "" to use the CPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Suppress some TF warnings
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    os.system("pip install sionna")
    import sionna

# Configure GPU
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Avoid warnings from TensorFlow
import warnings
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings('ignore')

# Other imports
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera

# Fix the seed for reproducible results
tf.random.set_seed(1)

pre_optimisation_cm = []
post_optimisation_cm = []
transmitter_coordinates = [[25,10,-2],[43.9,20.3,-2],[55.73, -14,3],[61.28, 40.13, 0], [75.94, 8.031, -0.7]]
final_orientations = []

## Configure the Scene and Generate Reference Data

In this example, we load the scene "etoile" which is already available in Sionna and then place a single transmitter **with trainable orientation** within the scene.


In [ ]:
for transmitter in transmitter_coordinates:

    print(transmitter)
    # Load the scene
    #scene = load_scene(sionna.rt.scene.etoile)
    # Load integrated scene
    scene = load_scene("/Users/andreamaestri/Desktop/tectwin/GEO/geoloc-rf/scenes/dusseldorf_tile_densification/dusseldorf_tile.xml") 
    scene.frequency = 3.5e9

    # Set the scattering coefficient of the radio material in the scene
    # to a non-zero value to enable scattering
        
    for mat in scene.radio_materials.values():
        mat.relative_permittivity=2.0
        mat.conductivity=5.0
        mat.scattering_coefficient=0.3
        mat.xpd_coefficient=0.1

    # Configure the transmit array
    scene.tx_array = PlanarArray(num_rows=1, num_cols=16,
                                vertical_spacing=0.5,
                                horizontal_spacing=0.5,
                                pattern="tr38901",
                                polarization="V",
                                polarization_model=2)

    # Configure the receive array (use to compute the coverage map)
    scene.rx_array = PlanarArray(num_rows=1, num_cols=1,
                                vertical_spacing=0.5,
                                horizontal_spacing=0.5,
                                pattern="iso",
                                polarization="V",
                                polarization_model=2)

    # Create a transmitter and add it to the scene


    tx = Transmitter(f"tx", position=transmitter,
                        orientation=tf.Variable([ 0.,0.,0.], tf.float32)) # Trainable orientation
    scene.add(tx)

    # Rectangle defining the target area
    target_center = np.array([0.,0.,0]) # Center
    target_size = np.array([300.,300.]) # Size
    target_orientation = np.array([0.1,0.1,0.1]) # Orientation: parallel to XY
    cm_cell_size = np.array([1.,1.]) # Each cell is 2mx2m

    with tf.GradientTape() as tape:
            
        # Compute coverage of the target area
        target_cm = scene.coverage_map(cm_center=target_center,
                                    cm_orientation=target_orientation, 
                                    cm_size=target_size, # Target area 
                                    cm_cell_size=cm_cell_size,
                                    diffraction=False, scattering=False, # Enable diffraction and scattering in addition to reflection and LoS
                                    check_scene=False) # Don't check the scene prior to compute to speed things up

        #scaling = 1e6
        #rate = tf.reduce_mean(tf.math.log(1. + ta
        #rget_cm.as_tensor()*scaling))/tf.math.log(2.)
        #loss = -rate
        loss = tf.reduce_mean(target_cm.as_tensor())
        
    # Compute gradients and apply through the optimizer
    grads = tape.gradient(loss, tape.watched_variables())

    cm = scene.coverage_map(num_samples=10e4, # Reduce if your GPU does not have enough memory
                        cm_cell_size=np.array([1.,1.]),
                        diffraction=True, scattering=True,  # Enables diffraction and scattering in addition to reflection and LoS
                        check_scene=False) # Don't check the scene prior to compute to speed things up)
    print(cm.as_tensor().shape)
    # Rectangle defining the target area
    target_center = np.array([50.,10.,-3]) # Center
    target_size = np.array([100.,100.]) # Size
    target_orientation = np.array([0.,0.,0.]) # Orientation: parallel to XY

    def visualize_target_area(ax, cell_size, center, size):
    # ax: Pyplot.Axes object on which to show the coverage map
    # cell_size : Size of the cells of the coverage map
    # center : Center of the target area
    # Size : Size of the target area
    
        def scene2cm(p):
            # Change coordinates p : (x,y) from scene to coverage map system
            shift = scene.center[:2].numpy() - scene.size[:2].numpy()*0.5
            p = p - shift
            p = np.floor(p/cell_size)
            return p

        # xy is the bottom left corner of the rectangle defining the target area
        xy = scene2cm(center[:2] - size[:2]*0.5)
        size_cm = np.floor(size/cell_size)
        rect = patches.Rectangle(xy, size_cm[0], size_cm[1], linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)


    # Configure an SGD optimizer
    # optimizer = tf.keras.optimizers.RMSprop(0.1)
    optimizer = tf.keras.optimizers.legacy.RMSprop(0.1)

    # Number of training steps
    num_steps = 1

    def train_step():
        """A single training step"""
        with tf.GradientTape() as tape:
            
            # Compute coverage of the target area
            target_cm = scene.coverage_map(cm_center=target_center,
                                        cm_orientation=target_orientation, 
                                        cm_size=target_size, # Target area 
                                        cm_cell_size=cm_cell_size,
                                        diffraction=True, scattering=True, # Enable diffraction and scattering in addition to reflection and LoS
                                        check_scene=False) # Don't check the scene prior to compute to speed things up
            # The loss function is a rate in bit
            # We fix an arbitrary scaling factor corresponding to the transmit to noise power ratio
            # The scaling has a direct impact the gradient magnitudes
            scaling = 1e6
            rate = tf.reduce_mean(tf.math.log(1. + target_cm.as_tensor()*scaling))/tf.math.log(2.)
            loss = -rate
        
        # Compute gradients and apply through the optimizer
        grads = tape.gradient(loss, tape.watched_variables())
        optimizer.apply_gradients(zip(grads, tape.watched_variables()))
        return rate

    for step in range(num_steps):       
        rate = train_step()
        print(f"Training step {step} - Rate: {rate.numpy():.2E} bit - tx orientation: {scene.transmitters[f'tx'].orientation.numpy()}", end='\r')


    cm_new = scene.coverage_map(num_samples=10e5, # Reduce if your GPU does not have enough memory
                            cm_cell_size=[1,1],
                            diffraction=True, scattering=True, # Enable diffraction and scattering in addition to reflection and LoS
                            check_scene=False) # Don't check the scene prior to compute to speed things up
    
    final_orientations.append(scene.transmitters[f'tx'].orientation.numpy())
    pre_optimisation_cm.append(cm)
    post_optimisation_cm.append(cm_new)

Next, we will compute and show a coverage map for the transmitter `"tx"`. The coverage map corresponds to the average received power into small rectangular regions (or *cells*) of a plane, which, by default, is parallel to the XY plane, and sized such that it covers the entire scene with an elevation of $1.5$ m.

The coverage map is divided into cells of size ``cm_cell_size`` (in meters). The smaller the size of the cells, the more precise the coverage map.

The shape of the coverage map is ``[num_tx, num_cells_y, num_cells_x]``

In the above figure, white areas are not hit by any ray. This usually implies that the coverage is negligible in these areas.

## Learn the Orientation of the Transmitter

We will now optimize through gradient descent the orientation of the transmitter to improve the coverage in an area defined by a rectangle with the following properties. Note that these properties are defined in the scene coordinate system.

Let's visualize the target area on the coverage map. We first define a utility function to highlight the target area.

Next, we optimize the coverage of the target area through gradient descent with respect to the orientation of the transmitter. To that end, during the optimization process, coverage is only computed for the target area.

Let's now compute the new coverage map over the entire scene with the optimized orientation.

Finally, the coverage map before and after optimization are shown, with the target area highlighed.

In [ ]:
# Display the initial coverage map
fig = cm.show(vmin=-150, vmax=-40)
fig.suptitle("Before")
# Highlight the target area
visualize_target_area(fig.axes[0], cm_cell_size, target_center, target_size,)

# Display the optimized coverage map
fig = cm_new.show(vmin=-150, vmax=-40)
fig.suptitle("After")
# Highlight the target area
visualize_target_area(fig.axes[0], cm_cell_size, target_center, target_size)

In [ ]:
for map in post_optimisation_cm:
    map.show()

In [ ]:
for map in pre_optimisation_cm:
    map.show()

We can also visualize the coverage maps as an overlay of the scene using the renderer.

In [ ]:
# Add a camera for viewing the scene from the sky
scene.add(Camera("top-view2", position=(0,0,300), orientation=(0.5*np.pi, 0.5*np.pi, 0.0)))

In [ ]:
fig = scene.render("top-view2")
fig.suptitle("No Coverage Map");

for i in range(2):# Rendering with the unoptimized orientation
    fig = scene.render("top-view2", coverage_map=pre_optimisation_cm[i], cm_vmin=-140, cm_vmax=-40)
    fig.suptitle("Before")

    # Rendering with the optimized orientation
    fig = scene.render("top-view2", coverage_map=post_optimisation_cm[i], cm_vmin=-140, cm_vmax=-40)
    fig.suptitle("After");

    print("\\n-------------------------------------------------\\n")

## Discussion

- Other loss functions could be used. For example, if multiple transmitters are considered, maximization of the signal-to-interference ratio could be used as objective function.

- Only the orientation is optimized in this notebook. However, other parameters could be learned as well, such as the geometry of the transmitter/receiver arrays as well as precoding vectors.

- In general, the optimization of a coverage map is a non-convex problem, especially when considering antenna arrays with complex radiation patterns. One could resort to other optimization methods such as Bayesian learning in this case.

In [ ]:
final_orientations

In [ ]:
"""[array([-0.57380253,  0.12531541, -0.01863085], dtype=float32),
 array([-0.4782467 ,  0.06706429,  0.04451044], dtype=float32),
 array([ 1.2872813 ,  0.20976876, -0.17728148], dtype=float32),
 array([ 0.19478539,  0.11065693, -0.17349428], dtype=float32),
 array([ 2.0597959 ,  0.1899164 , -0.06186479], dtype=float32)]
"""